# InfoBridge on high-dimensional MI data

This notebook implements High MI Benchmark experiment from Section 5.4 in [paper](https://arxiv.org/pdf/2502.01383)

Benchmark is taken from [mutinfo library](https://github.com/VanessB/mutinfo)


In [ ]:

import torch
import lightning.pytorch as pl


from mutinfo.distributions.base import CorrelatedNormal, CorrelatedUniform, SmoothedUniform
from infobridge import InfoBridge

pl.seed_everything(42)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')


Seed set to 42


Using device: cuda


## Parameters of model and MI estimation problem


`task_name` - the random variable type. Options: 'gaussian', 'corr_uniform', 'smoothed_uniform'

`mi_gt` - ground truth MI

`dim` - dimensionality of random varible.

`n_train` - number of samples in train dataset

`n_eval` - number of samples in validation dataset

`eps` - stochasticity parameter of InfoBridge, `eps=0.01` is recommended for this problem. For additional information see the [paper](https://arxiv.org/pdf/2502.01383)


In [2]:
task_name = 'gaussian'  # options: 'gaussian', 'corr_uniform', 'smoothed_uniform'
mi_gt = 80
dim = 160
n_train = 100000
n_eval = 10000

# InfoBridge parameters
eps = 0.01
lr = 3e-4
ema_decay = 0.999
batch_size = 128
max_epochs = 30
val_epoch_freq = 1
mc_iters_eval = 1


In [3]:

if dim <= 5:
    time_embed = 64
    n_filters = 64
elif dim <= 25:
    time_embed = 128
    n_filters = 128
else:
    time_embed = 256
    n_filters = 256
    
print(f'task={task_name}, dim={dim}, mi_gt={mi_gt}')
print(f'UnetMLP params: n_filters={n_filters}, time_embed={time_embed}')

task=gaussian, dim=160, mi_gt=80
UnetMLP params: n_filters=256, time_embed=256


## Construct the random variables (X, Y) with known MI

In [4]:
if task_name == 'gaussian':
    task = CorrelatedNormal(
        mi_gt, dim, dim, randomize_interactions=False, shuffle_interactions=True
    )
elif task_name == 'corr_uniform':
    task = CorrelatedUniform(
        mi_gt, dim, dim, randomize_interactions=False, shuffle_interactions=True
    )
elif task_name == 'smoothed_uniform':
    task = SmoothedUniform(mi_gt, dim, randomize_interactions=False)
else:
    raise ValueError(f'Unsupported task_name: {task_name}')

x_train_np, y_train_np = task.rvs(n_train)
x_eval_np, y_eval_np = task.rvs(n_eval)

x0_train = torch.tensor(x_train_np, dtype=torch.float32)
x1_train = torch.tensor(y_train_np, dtype=torch.float32)
x0_eval = torch.tensor(x_eval_np, dtype=torch.float32)
x1_eval = torch.tensor(y_eval_np, dtype=torch.float32)

print(f'Train dataset shape: x0 {tuple(x0_train.shape)}, x1 {tuple(x1_train.shape)}')
print(f'Validation dataset shape: x0 {tuple(x0_eval.shape)}, x1 {tuple(x1_eval.shape)}')

Train dataset shape: x0 (100000, 160), x1 (100000, 160)
Validation dataset shape: x0 (10000, 160), x1 (10000, 160)


## Define UnetMLP backbone and InfoBridge module

In [5]:
import torch
from torch import nn
from functools import partial

def exists(x):
    return x is not None

def default(val, d):
    if exists(val):
        return val
    return d() if callable(d) else d

class Block(nn.Module):
    def __init__(self, dim, dim_out, groups=8, shift_scale=True):
        super().__init__()
        self.proj = nn.Linear(dim, dim_out)
        self.act = nn.SiLU()
        self.norm = nn.GroupNorm(groups, dim)
        self.shift_scale = shift_scale

    def forward(self, x, t=None):
        x = self.norm(x)
        x = self.act(x)
        x = self.proj(x)
        if exists(t):
            if self.shift_scale:
                scale, shift = t
                x = x * (scale.squeeze() + 1) + shift.squeeze()
            else:
                x = x + t
        return x

class ResnetBlock(nn.Module):
    def __init__(self, dim, dim_out, *, time_emb_dim=None, groups=32, shift_scale=False):
        super().__init__()
        self.shift_scale = shift_scale
        self.mlp = nn.Sequential(
            nn.SiLU(),
            nn.Linear(time_emb_dim, dim_out * 2 if shift_scale else dim_out),
        ) if exists(time_emb_dim) else None
        self.block1 = Block(dim, dim_out, groups=groups, shift_scale=shift_scale)
        self.block2 = Block(dim_out, dim_out, groups=groups, shift_scale=shift_scale)
        self.lin_layer = nn.Linear(dim, dim_out) if dim != dim_out else nn.Identity()

    def forward(self, x, time_emb=None):
        scale_shift = None
        if exists(self.mlp) and exists(time_emb):
            time_emb = self.mlp(time_emb)
            scale_shift = time_emb
        h = self.block1(x, t=scale_shift)
        h = self.block2(h)
        return h + self.lin_layer(x)

class UnetMLP_simple(nn.Module):
    def __init__(
        self,
        dim,
        init_dim=128,
        dim_mults=(),
        resnet_block_groups=8,
        time_dim=128,
        nb_var=2,
    ):
        super().__init__()
        self.nb_var = nb_var
        init_dim = default(init_dim, dim)
        dims = [init_dim, *map(lambda m: init_dim * m, dim_mults)]
        in_out = list(zip(dims[:-1], dims[1:]))
        block_klass = partial(ResnetBlock, groups=resnet_block_groups)

        self.init_lin = nn.Linear(dim * 2, init_dim)
        self.time_mlp = nn.Sequential(
            nn.Linear(nb_var, time_dim),
            nn.GELU(),
            nn.Linear(time_dim, time_dim),
        )

        self.downs = nn.ModuleList([])
        self.ups = nn.ModuleList([])

        for dim_in, _ in in_out:
            self.downs.append(nn.ModuleList([block_klass(dim_in, dim_in, time_emb_dim=time_dim)]))

        mid_dim = dims[-1]
        self.mid_block1 = block_klass(mid_dim, mid_dim, time_emb_dim=time_dim)

        for dim_in, dim_out in reversed(in_out):
            self.ups.append(nn.ModuleList([block_klass(dim_out + dim_in, dim_out, time_emb_dim=time_dim)]))

        self.final_res_block = block_klass(init_dim * 2, init_dim, time_emb_dim=time_dim)
        self.proj = nn.Linear(init_dim, dim)
        self.proj.weight.data.fill_(0.0)
        self.proj.bias.data.fill_(0.0)
        self.final_lin = nn.Sequential(
            nn.GroupNorm(resnet_block_groups, init_dim),
            nn.SiLU(),
            self.proj,
        )

    def forward(self, x, t, std=None):
        t = t.reshape(t.size(0), self.nb_var)
        x = self.init_lin(x.float())
        r = x.clone()
        t = self.time_mlp(t).squeeze()
        h = []
        for blocks in self.downs:
            x = blocks[0](x, t)
            h.append(x)
        for blocks in self.ups:
            x = torch.cat((x, h.pop()), dim=1)
            x = blocks[0](x, t)
        x = torch.cat((x, r), dim=1)
        x = self.final_res_block(x, t)
        out = self.final_lin(x)
        if std is not None:
            return out / std
        return out

class CondMLPTimeMINDE(nn.Module):
    """MINDE-style conditional MLP adapted to InfoBridge backbone interface."""

    def __init__(self, minde_mlp):
        super().__init__()
        self.net = minde_mlp

    def forward(self, x, t, class_labels=None):
        if class_labels is None:
            class_labels = torch.ones_like(t)
        time_features = torch.stack([t, class_labels], dim=1)
        return self.net(x, time_features)
        

In [6]:
base_net = UnetMLP_simple(
    dim=dim,
    init_dim=n_filters,
    dim_mults=(),
    time_dim=time_embed,
    nb_var=2,
).to(device)

cond_backbone = CondMLPTimeMINDE(base_net).to(device)

infobridge_model = InfoBridge(
    backbone=cond_backbone,
    eps=eps,
    lr=lr,
    t_eps=1e-6,
    nfe_posterior=100,
    base_mc_iters=1,
    default_batch_size=batch_size,
    ema_decay=ema_decay,
).to(device)



## Train InfoBridge

In [7]:

infobridge_model.fit(
    x0_train,
    x1_train,
    x0_val=x0_eval,
    x1_val=x1_eval,
    max_epochs=max_epochs,
    trainer_kwargs={
        'check_val_every_n_epoch': val_epoch_freq, 'enable_progress_bar': True, 
    'num_sanity_val_steps': 0,
    },
);


/trinity/home/sergei.kholkin/.conda/envs/tabdiff/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /trinity/home/sergei.kholkin/.conda/envs/tabdiff/lib ...
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
/trinity/home/sergei.kholkin/.conda/envs/tabdiff/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `l

┏━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name       ┃ Type             ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ backbone   │ CondMLPTimeMINDE │  784 K │ train │     0 │
│ 1 │ drift_plan │ DriftWrapper     │  784 K │ train │     0 │
│ 2 │ drift_ind  │ DriftWrapper     │  784 K │ train │     0 │
│ 3 │ bm_plan    │ BridgeMatching   │  784 K │ train │     0 │
│ 4 │ bm_ind     │ BridgeMatching   │  784 K │ train │     0 │
└───┴────────────┴──────────────────┴────────┴───────┴───────┘

Trainable params: 784 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 784 K                                                                                                
Total estimated model params size (MB): 3                                                                          
Modules in train mode: 43                                                                                          
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

/trinity/home/sergei.kholkin/.conda/envs/tabdiff/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:434: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.
/trinity/home/sergei.kholkin/.conda/envs/tabdiff/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:434: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


Epoch 29: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 782/782 [00:07<00:00, 102.66it/s, v_num=82, val_mi=79.00, loss_total=7.970]

`Trainer.fit` stopped: `max_epochs=30` reached.


Epoch 29: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 782/782 [00:07<00:00, 102.10it/s, v_num=82, val_mi=79.00, loss_total=7.970]


In [8]:
infobridge_model.to(device);

## Estimate mutual information

Increase `mc_iters` to improve stability.

In [9]:
with torch.no_grad():
    mi_est = infobridge_model.estimate_mi(x0_eval, x1_eval, mc_iters=mc_iters_eval)

print(f'Ground-truth MI: {mi_gt}')
print(f'Estimated MI (InfoBridge): {mi_est.item():.4f}')

Ground-truth MI: 80
Estimated MI (InfoBridge): 81.1674


## Optional posterior sampling sanity check

In [10]:
with torch.no_grad():
    x0_vis = x0_eval[:8].to(device)
    x1_samples = infobridge_model(x0_vis).cpu()

print('x0 sample shape:', tuple(x0_vis.shape))
print('generated x1 shape:', tuple(x1_samples.shape))
print('mean |x1_sample - x0|:', (x1_samples - x0_vis.cpu()).abs().mean().item())

x0 sample shape: (8, 160)
generated x1 shape: (8, 160)
mean |x1_sample - x0|: 1.133078694343567
